In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

print("Loading preprocessed data...")
df = pd.read_csv('preprocessed_data_final.csv')

# Assuming your target column is 'target'. If it's different, change it here:
target_column = 'Target'

# 2. Prepare the data
print("\nPreparing data for modeling...")
X = df.drop(columns=[target_column])
y = df[target_column]

"""
# Reduce dimensionality
pca = PCA(n_components=117)
X_reduced = pca.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.3, stratify=y, random_state=42
)
"""

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Compute class weights manually
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Build model
model = Sequential([
    Dense(128, input_dim=X.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Fit
early_stop = EarlyStopping(patience=10, restore_best_weights=True)

model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=8,
    class_weight=class_weight_dict,
    callbacks=[early_stop],
    verbose=1
)

# Predict and evaluate
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Loading preprocessed data...

Preparing data for modeling...
Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.5402 - loss: 1.1314 - val_accuracy: 0.5833 - val_loss: 0.5997
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6148 - loss: 0.5702 - val_accuracy: 0.9583 - val_loss: 0.3969
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6045 - loss: 0.7217 - val_accuracy: 0.9583 - val_loss: 0.2938
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8042 - loss: 0.3193 - val_accuracy: 1.0000 - val_loss: 0.2505
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7437 - loss: 0.4228 - val_accuracy: 1.0000 - val_loss: 0.2304
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7100 - loss: 0.4265 - val_accuracy: 1.0000 - val_loss: 0.2169
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7167 - loss: 0.3361 - val_accuracy: 1.0000 - val_loss: 0.2103
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11m